In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 1. Load Module
from selenium import webdriver 
from bs4 import BeautifulSoup 
import urllib.request as req 
import pandas as pd 
import numpy as np 
import collections as co 
import time 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

# 2. Enter the website - Flo
url = 'https://play.google.com/store/apps/details?id=com.towneers.www&showAllReviews=true'
# driverPath = "C:/Users/vitri/Downloads/chromedriver.exe" 

# options = webdriver.ChromeOptions()
# options.add_experimmental_option('excludeSwitches',['enable-logging'])
# driver = webdriver.Chrome(driverPath)
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.maximize_window()
# while(True):
#     pass


# driver = webdriver.Chrome(driverPath) 
# driver.get(url)

# 3. Scrape Data 
SCROLL_PAUSE_TIME = 1.5 
last_height = driver.execute_script("return document.body.scrollHeight") 
# for i in range(5):
while True:
    #6번의 스크롤링
    for i in range(50):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(SCROLL_PAUSE_TIME)
    
    #더보기 클릭
#     driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()

    #종료 조건
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

## [전체 리뷰] 버튼 클릭하여 펼치기
spread_review = driver.find_elements_by_xpath("//button[@jsaction='click:TiglPc']")

for i in range(len(spread_review)):
    isTrue = spread_review[i].is_displayed()
    print('Element is visible? '+str(isTrue))
    if isTrue:
        spread_review[i].send_keys(Keys.ENTER)
        print(str(i)+"th more button is clicked and wait 1.5 secs...") 
        time.sleep(SCROLL_PAUSE_TIME)
    
reviews = driver.find_elements_by_xpath("//span[contains(@jsname, 'bN97Pc')]")
for i in range(len(reviews)): 
    print(str(i) + "\t" + reviews[i].text)

long_reviews = driver.find_elements_by_xpath("//span[@jsname='fbQN7e']")
for i in range(len(reviews)): 
    print(long_reviews[i].text)

## merge two list 
merged_review = [t.text if t.text!='' else long_reviews[i].text for i,t in enumerate(reviews)]

dates = driver.find_elements_by_xpath("//div[@class='bAhLNe kx8XBd']/div/span[@class='p2TkOb']")
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']")
stars_text = stars[3].get_attribute('aria-label')

# 4. Merge Data 
res_deque = co.deque([]) 
for i in range(len(reviews)): 
    res_deque.append({ 
        'DATE' : dates[i].text, 
        'STAR' : stars[i].get_attribute('aria-label'), 
        'LIKE' : likes[i].text, 
        'REVIEW' : merged_review[i] 
        })

res_df = pd.DataFrame(res_deque) 
res_df.to_csv('C:/Users/vitri/Downloads/Crawling_data.csv', index=False, encoding='utf-8-sig')




====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\vitri\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache
